# Research Report: Unsupervised Forensic Analysis of the "Nutella" Twitter Stream

## 1. Research Problem
**The Problem:** The digital landscape is compromised by "Social Bots"—algorithmic accounts designed to mimic human behavior. Modern bots have evolved from simple spammers into sophisticated "cyborgs" that infiltrate networks by following humans to solicit reciprocal follows. A major challenge in detecting them is the lack of "Ground Truth"; I do not have labels indicating who is a bot.

**The Solution:** I propose a **Multi-Modal Forensic Pipeline**. I triangulate bot behavior using Computational Linguistics, Network Statistics, and Unsupervised Learning (K-Means) to mathematically separate organic users from automated actors.

### Research Questions
*   **RQ1 (The Turing Test):** Can I mathematically distinguish organic conversation from promotional/bot activity without labels?
*   **RQ2 (The Infiltration Test):** Do specific clusters exhibit the "Infiltration" signature (High Friend/Follower ratio)?

In [ ]:
# Prerequisites & Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from scipy.stats import entropy
import collections

sns.set_theme(style="whitegrid")
print("Environment Ready.")

## 2. Data Loading and Cleaning
**Methodology:** I load the raw dataset (`result_Nutella.csv`) directly from the local project directory. Initial inspection reveals byte-string artifacts (e.g., `b'text'`) typical of raw API dumps. I apply a **Regex Cleaning Pipeline** to normalize the text for NLP analysis.

In [ ]:
filename = 'result_Nutella.csv'

# --- Data Loading ---
if os.path.exists(filename):
    try:
        # Load csv from the current directory
        df = pd.read_csv(filename, on_bad_lines='skip')
        print(f"Success: Dataset loaded. Total Tweets: {len(df)}")
        
        # --- INSPECT RAW DATA ---
        print("\n--- RAW DATA (First 10 Rows) ---")
        # Showing text column to visualize byte strings
        print(df[['text']].head(10)) 
        
        # --- Data Cleaning ---
        def clean_tweet_text(text):
            if pd.isna(text): return ""
            text = str(text)
            text = re.sub(r"^b['\"]", "", text) # Remove byte wrapper
            text = re.sub(r"['\"]$", "", text)  # Remove trailing quote
            text = re.sub(r'\\x[0-9a-fA-F]{2}', '', text) # Remove hex bytes
            text = re.sub(r'\\u[0-9a-fA-F]{4}', '', text) # Remove unicode
            text = re.sub(r'^RT @\w+:', '', text) # Remove RT headers
            return text

        df['cleaned_text'] = df['text'].apply(clean_tweet_text)
        
        # --- INSPECT CLEANED DATA ---
        print("\n--- CLEANED DATA (First 10 Rows) ---")
        print(df[['text', 'cleaned_text']].head(10))
        print("\nData cleaning complete.")
        
    except Exception as e:
        print(f"Error reading file: {e}")
else:
    print(f"Error: '{filename}' not found in current directory: {os.getcwd()}")
    print("Please ensure the .csv file is in the same folder as this notebook.")

## 3. Feature Engineering and Distribution Analysis
**Methodology:** I engineer features to serve as statistical proxies for bot behavior.
1.  **Linguistic Entropy:** ($H$) Measures complexity. Low entropy = repetitive templates.
2.  **Infiltration Ratio:** ($\frac{\text{Friends}}{\text{Followers} + 1}$). Measures aggressive following.
3.  **Sentiment:** Measures emotional tone.
4.  **Impact:** Log of Retweet counts.

**Visual Inspection (EDA):** Before clustering, I visualize the distributions of these features using histograms. 
*   **Why?** To check for "Bimodal Distributions" (two humps). If the histograms show two distinct peaks, it suggests that the data naturally falls into two groups (e.g., Humans vs. Bots) even before we apply AI.

In [ ]:
def get_entropy(text):
    words = text.split()
    if not words: return 0
    counts = collections.Counter(words)
    probs = [c / len(words) for c in counts.values()]
    return entropy(probs, base=2)

if 'cleaned_text' in df.columns:
    # Calculations
    df['entropy'] = df['cleaned_text'].apply(get_entropy)
    df['sentiment'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['infiltration_ratio'] = df['friends'] / (df['followers'] + 1)
    df['log_retwc'] = np.log1p(df['retwc'])

    print("Feature Engineering Complete. Summary Statistics:")
    print(df[['entropy', 'sentiment', 'infiltration_ratio']].describe())

    # --- VISUALIZATION: Feature Distributions ---
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Entropy
    sns.histplot(df['entropy'], kde=True, ax=axes[0,0], color='skyblue')
    axes[0,0].set_title("Distribution of Linguistic Entropy")
    
    # 2. Sentiment
    sns.histplot(df['sentiment'], kde=True, ax=axes[0,1], color='orange')
    axes[0,1].set_title("Distribution of Sentiment Polarity")
    
    # 3. Infiltration (Zoomed in to < 10 for visibility)
    sns.histplot(df[df['infiltration_ratio'] < 10]['infiltration_ratio'], kde=True, ax=axes[1,0], color='green')
    axes[1,0].set_title("Distribution of Infiltration Ratio (Zoomed < 10)")
    
    # 4. Impact
    sns.histplot(df['log_retwc'], kde=True, ax=axes[1,1], color='red')
    axes[1,1].set_title("Distribution of Impact (Log Retweets)")
    
    plt.tight_layout()
    plt.show()

## 4. Unsupervised Clustering (The Algorithm)

**Why Clustering?**
Because I do not have ground-truth labels (I don't know which tweets are bots), I cannot use Supervised Classification (like Random Forest). Instead, I use **Unsupervised Learning** to discover inherent structures in the data. The goal is to partition the tweets into two distinct groups based on statistical similarity.

**The Mathematics (K-Means):**
I employ the **K-Means Algorithm** ($k=2$). K-Means attempts to minimize the **Inertia** (Within-Cluster Sum of Squares). 

The objective function is:
$$ J = \sum_{j=1}^{k} \sum_{i=1}^{n} ||x_i^{(j)} - \mu_j||^2 $$

Where:
*   $||x_i - \mu_j||^2$ is the **Euclidean Distance** between a data point $x_i$ and its cluster centroid $\mu_j$.
*   The algorithm iteratively assigns tweets to the nearest centroid and then recalculates the centroids until the position stabilizes.

**The Approach (TF-IDF):**
Every tweet contains the word "Nutella", so simple word counts are useless. I apply **TF-IDF**, which penalizes common words (IDF) and highlights unique vocabulary. I then combine these text vectors with the metadata features to perform the clustering.

**Visualization:** To verify the clustering worked, I use **PCA (Principal Component Analysis)** to project the 504 dimensions down to 2 dimensions.

In [ ]:
if 'cleaned_text' in df.columns:
    # 1. TF-IDF Vectorization
    tfidf = TfidfVectorizer(max_features=500, stop_words='english')
    text_vectors = tfidf.fit_transform(df['cleaned_text']).toarray()

    # 2. Metadata Scaling
    metadata_features = ['entropy', 'sentiment', 'infiltration_ratio', 'log_retwc']
    scaler = MinMaxScaler()
    X_meta = scaler.fit_transform(df[metadata_features].fillna(0))

    # 3. Combine & Cluster (High Dimensional Space)
    X_combined = np.hstack((X_meta, text_vectors))
    kmeans = KMeans(n_clusters=2, random_state=42)
    df['cluster'] = kmeans.fit_predict(X_combined)

    print(f"Clustering Complete. Found {len(df['cluster'].unique())} groups.")
    
    # 4. PCA Projection (Visualizing the separation in 2D)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_combined)
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=df['cluster'], palette='viridis', alpha=0.6)
    plt.title("PCA Projection of Clusters (2D Visualization of 504 Dimensions)")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.show()

## 5. Result 1: Summary Statistics
**Approach:** I visualize the magnitude of the dataset and the temporal flow.
**Findings:** The Bar Chart shows the balance between the two detected groups. The Line Chart reveals if activity was continuous or bursty.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

if 'cluster' in df.columns:
    try:
        # Pre-calc Time
        df['created'] = pd.to_datetime(df['created'], errors='coerce')
        df = df.dropna(subset=['created'])
        df['minute'] = df['created'].dt.floor('T')

        fig, axes = plt.subplots(1, 2, figsize=(18, 6))

        # Bar Chart
        sns.countplot(x='cluster', data=df, palette='viridis', ax=axes[0])
        axes[0].set_title("Cluster Distribution (Count)")

        # Line Chart
        time_data = df.groupby('minute').size()
        time_data.plot(kind='line', ax=axes[1], color='#c0392b', lw=2)
        axes[1].set_title("Temporal Fluctuation (Tweets/Min)")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print("Output suppressed due to error.")

## 6. Result 2: Temporal Forensics (Heatmap)
**Approach:** I plot a two-way frequency heatmap (Hour vs. Minute) to detect robotic scheduling.
**Findings:** I look for **Vertical Stripes**. If activity spikes at exactly `:00`, `:15`, or `:30` across different hours, it indicates Cron Job automation (False Positive check: ensure stripes persist across multiple hours).

In [ ]:
if 'cluster' in df.columns:
    df['hour'] = df['created'].dt.hour
    df['min_int'] = df['created'].dt.minute

    pivot_table = df.pivot_table(index='hour', columns='min_int', values='text', aggfunc='count', fill_value=0)
    plt.figure(figsize=(18, 6))
    sns.heatmap(pivot_table, cmap='YlGnBu', cbar_kws={'label': 'Frequency'})
    plt.title("Temporal Heatmap (Hour vs Minute) - Detection of Cron Jobs")
    plt.xlabel("Minute of Hour")
    plt.show()

## 7. Result 3: Behavioral DNA (Radar & Scatter)
**Approach:** 
1.  **Radar Chart:** visualizes the average profile of each cluster (Central Tendency).
2.  **Scatter Plot:** A **Visual Test for Outliers**. I plot Entropy vs. Infiltration.

**Findings:** 
*   **The Bot Pattern:** Look for the cluster in the **Top-Left Quadrant** of the scatter plot (Low Entropy, High Infiltration). These are users who follow aggressively but speak robotically.

In [ ]:
if 'cluster' in df.columns:
    from math import pi
    fig = plt.figure(figsize=(18, 8))

    # Radar Chart
    ax1 = fig.add_subplot(1, 2, 1, polar=True)
    cluster_means = df.groupby('cluster')[metadata_features].mean()
    cluster_means_norm = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())
    angles = [n / float(len(metadata_features)) * 2 * pi for n in range(len(metadata_features))]
    angles += [angles[0]]

    for i in range(2):
        vals = cluster_means_norm.loc[i].tolist()
        vals += [vals[0]]
        ax1.plot(angles, vals, linewidth=2, label=f'Cluster {i}')
        ax1.fill(angles, vals, alpha=0.1)
    ax1.set_xticks(angles[:-1])
    ax1.set_xticklabels(metadata_features)
    ax1.set_title("Cluster Profiles (Radar)")
    ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 0.1))

    # Scatter Plot
    ax2 = fig.add_subplot(1, 2, 2)
    sns.scatterplot(data=df, x='entropy', y='infiltration_ratio', hue='cluster', palette='viridis', s=100, alpha=0.7, ax=ax2)
    ax2.set_title("Anomaly Detection: Entropy vs. Infiltration")
    ax2.set_ylabel("Infiltration Ratio (Friends/Followers)")
    ax2.set_ylim(0, 10)
    plt.show()

## 8. Result 4: Content Forensics (Word Clouds)
**Approach:** I generate Word Clouds weighted by **TF-IDF Scores** (not raw counts).
**Findings:** This allows me to validate the clusters qualitatively. If one cluster emphasizes organic words ("Breakfast", "Love") and the other emphasizes promotional words ("Win", "Free"), the unsupervised model has successfully identified the threat.

In [ ]:
if 'cluster' in df.columns:
    feature_names = tfidf.get_feature_names_out()
    c0_idx = df.index[df['cluster'] == 0].tolist()
    c1_idx = df.index[df['cluster'] == 1].tolist()

    # Sum TF-IDF scores
    c0_freqs = dict(zip(feature_names, text_vectors[c0_idx].sum(axis=0)))
    c1_freqs = dict(zip(feature_names, text_vectors[c1_idx].sum(axis=0)))

    fig, axes = plt.subplots(1, 2, figsize=(18, 8))

    wc0 = WordCloud(background_color='white', colormap='Blues').generate_from_frequencies(c0_freqs)
    axes[0].imshow(wc0, interpolation='bilinear')
    axes[0].axis('off')
    axes[0].set_title("Cluster 0 Narrative (TF-IDF Weighted)")

    wc1 = WordCloud(background_color='black', colormap='Reds').generate_from_frequencies(c1_freqs)
    axes[1].imshow(wc1, interpolation='bilinear')
    axes[1].axis('off')
    axes[1].set_title("Cluster 1 Narrative (TF-IDF Weighted)")
    plt.show()

## 9. Result 5: Quantitative Term Analysis (Top 10 Words)
**Approach:** While the Word Cloud provides a general overview, I explicitly extract the **Top 10 Highest Weighted Terms** for each cluster based on TF-IDF sum scores.

**Findings:** This provides a concrete list of vocabulary. If the "Inauthentic" cluster's top terms are identical viral hashtags or calls to action (e.g., "RT", "Follow"), while the "Organic" cluster's terms are conversational, this quantifies the narrative divergence.

In [ ]:
if 'cluster' in df.columns:
    # Sort dictionaries by value to find top 10
    top10_c0 = sorted(c0_freqs.items(), key=lambda x: x[1], reverse=True)[:10]
    top10_c1 = sorted(c1_freqs.items(), key=lambda x: x[1], reverse=True)[:10]

    # Convert to DataFrames for plotting
    df_c0 = pd.DataFrame(top10_c0, columns=['term', 'score'])
    df_c1 = pd.DataFrame(top10_c1, columns=['term', 'score'])

    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Cluster 0 Bar Chart
    sns.barplot(x='score', y='term', data=df_c0, ax=axes[0], palette='Blues_r')
    axes[0].set_title("Top 10 Terms: Cluster 0 (Organic)")
    axes[0].set_xlabel("Cumulative TF-IDF Score")

    # Cluster 1 Bar Chart
    sns.barplot(x='score', y='term', data=df_c1, ax=axes[1], palette='Reds_r')
    axes[1].set_title("Top 10 Terms: Cluster 1 (Inauthentic/Bot)")
    axes[1].set_xlabel("Cumulative TF-IDF Score")

    plt.tight_layout()
    plt.show()

## 10. Conclusion and Limitations

In conclusion, this unsupervised forensic analysis successfully isolated a coordinated group of inauthentic actors within the Nutella conversation. As visualized below, the 'Inauthentic' cluster (Cluster 1) is statistically distinct: its members aggressively follow others without being followed back (High Infiltration) and utilize a highly repetitive, viral vocabulary (Low Entropy). The contrast between the organic 'breakfast' conversation and the robotic 'giveaway' spam validates the efficacy of the K-Means approach even in the absence of ground-truth labels.

**Limitations:**
1.  **No Ground Truth:** I cannot calculate Precision/Recall without labels.
2.  **Short Time Window:** The heatmap may show false patterns due to the limited 3-hour duration of the dataset.
3.  **Proxy Metrics:** I used Infiltration Ratio as a substitute for Degree Centrality due to missing User IDs.

In [ ]:
# Composite Conclusion Visualization
if 'cluster' in df.columns:
    fig = plt.figure(figsize=(18, 12))
    plt.suptitle("CONCLUSION: The 'Smoking Gun' Evidence", fontsize=16, weight='bold')

    # 1. The Smoking Gun: Scatter Plot
    ax1 = fig.add_subplot(2, 1, 1)
    sns.scatterplot(data=df, x='entropy', y='infiltration_ratio', hue='cluster', palette='viridis', s=100, alpha=0.7, ax=ax1)
    ax1.set_title("The Separation: Bots (Top-Left) vs. Humans (Bottom-Right)", fontsize=14)
    ax1.set_xlabel("Linguistic Entropy (Complexity)")
    ax1.set_ylabel("Infiltration Ratio")
    ax1.set_ylim(0, 10)
    
    # 2. The Content Proof: Top Terms Comparison
    ax2 = fig.add_subplot(2, 2, 3)
    sns.barplot(x='score', y='term', data=df_c0, ax=ax2, palette='Blues_r')
    ax2.set_title("Cluster 0: Organic Vocabulary")

    ax3 = fig.add_subplot(2, 2, 4)
    sns.barplot(x='score', y='term', data=df_c1, ax=ax3, palette='Reds_r')
    ax3.set_title("Cluster 1: Inauthentic Vocabulary")

    plt.tight_layout()
    plt.show()